<a href="https://colab.research.google.com/github/ClickerDG/datamining/blob/master/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Лабораторная работа №2, Козлов КС-43

Аутентифицируем юзера

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Сеттим линку на цсв файл

In [ ]:
link = 'https://drive.google.com/file/d/1TLC2_HoNcH-d5gpqQSC_-J2s-BAztYb2/view?usp=sharing' 
linkPart, trash = link.split('/view')
trash, id = linkPart.split('/d/')
print (id)

1TLC2_HoNcH-d5gpqQSC_-J2s-BAztYb2


Качаем файлик в колаб сторэдж

In [ ]:
import pandas as pd

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('sms-spam-corpus.csv')  
csvDataFrame = pd.read_csv('sms-spam-corpus.csv', encoding='latin1', )

Считаем P(ham) и P(spam)

In [ ]:
p_ham = csvDataFrame[csvDataFrame['v1'] == 'ham'].count().v1 / csvDataFrame.count().v1
print (p_ham)
p_spam = csvDataFrame[csvDataFrame['v1'] == 'spam'].count().v1 / csvDataFrame.count().v1
print (p_spam)


0.8659368269921034
0.13406317300789664


In [ ]:
import re
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

stemmer = SnowballStemmer('english')

stopWords = ['', 'am', 'is', 'are', 'be', 'being', 'been', 'have', 'has', 'having', 'had', 'do', 'does', 'doing', 'did']
csvDataFrame['v2'].replace(regex=True, inplace=True, to_replace=r'[^a-zA-Z\s]', value=r'')
csvDataFrame['v2'].replace(regex=True, inplace=True, to_replace=stopWords, value=r'')

def stemming(content):
  words = word_tokenize(content)
  stemmedWords = [stemmer.stem(word) for word in words]
  sentence = ' '.join(stemmedWords)
  return sentence


csvDataFrame['v2'] = csvDataFrame['v2'].map(lambda content: content.lower()).astype(str)
csvDataFrame['v2'] = csvDataFrame['v2'].map(lambda content: stemming(content)).astype(str)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from collections import defaultdict

isSpam = csvDataFrame['v1'] == 'spam'
spam = csvDataFrame[isSpam]['v2']
spamDictionary = defaultdict(int)

for message in spam:
  words = word_tokenize(message)
  for word in words:
    spamDictionary[word] += 1

In [ ]:
isHam = csvDataFrame['v1'] == 'ham'
ham = csvDataFrame[isHam]['v2']
hamDictionary = defaultdict(int)
for message in ham:
  words = word_tokenize(message)
  for word in words:
    hamDictionary[word] += 1

In [ ]:
globalDictionary = defaultdict(int)

for dictionary in (spamDictionary, hamDictionary) :
  for key, value in dictionary.items() :
    globalDictionary[key] += int(value)

print(len(globalDictionary), len(spamDictionary), len(hamDictionary))

7113 2041 6043


Определяем функции для p(word, ham) и p(word, spam) и p(bodytext, ham), p(bodytext, spam)

In [ ]:
import numpy as np

def p_word_ham(word) :
  return int(hamDictionary[word]) / len(hamDictionary.keys())

def p_word_spam(word) :
  return int(spamDictionary[word]) / len(spamDictionary.keys())

def p_bodytext_ham(bodytext) :
  words = set(str.split(bodytext))
  return np.prod(np.array([p_word_ham(word) for word in words]))

def p_bodytext_spam(bodytext) :
  words = set(str.split(bodytext))
  return np.prod(np.array([p_word_spam(word) for word in words]))

def p_ham_bodytext(bodytext) :
  return p_ham * p_bodytext_ham(bodytext)

def p_spam_bodytext(bodytext) :
  return p_spam * p_bodytext_spam(bodytext)

def process_words(bodytext):
    for word in text.split():
        hamDictionary[word] += 1
        spamDictionary[word] += 1
        globalDictionary[word] += 1

def calculate_probabilities(message):
  process_words(message)
  p_ham_message = p_ham_bodytext(message)
  p_spam_message = p_spam_bodytext(message)

  pham = p_ham_message / (p_ham_message + p_spam_message)
  pspam = p_spam_message / (p_ham_message + p_spam_message)

  print(f'p_ham_bodytext : {pham:.10}%')
  print(f'p_spam_bodytext : {pspam:.10}%')

In [ ]:
print(csvDataFrame.v2[0])
calculate_probabilities(csvDataFrame.v2[0])  

go until jurong point crazi avail onli in bug n great world la e buffet cine there got ore wat
p_ham_bodytext : 0.9999999108%
p_spam_bodytext : 8.920917967e-08%


Вероятности для всех сообщений


In [ ]:
def calculate_probabilities_for_messages(messages) :
  for message in messages:
    print(message)
    calculate_probabilities(message)  

calculate_probabilities_for_messages(csvDataFrame.v2)

Выходные данные были обрезаны до нескольких последних строк (5000).
p_spam_bodytext : 0.02169634291%
no that just mean you a fat head
p_ham_bodytext : 0.99087186%
p_spam_bodytext : 0.009128139973%
sound like a plan cardiff still here and still cold im sit on the radiat
p_ham_bodytext : 0.999292697%
p_spam_bodytext : 0.0007073030118%
serious what like proper tongu her
p_ham_bodytext : 0.9874893151%
p_spam_bodytext : 0.01251068493%
shes good she was wonder if you wont say hi but shes smile now so how you cope with the long dtanc
p_ham_bodytext : 0.999942803%
p_spam_bodytext : 5.719702924e-05%
how i noe shes in da car now later then c lar im wear short
p_ham_bodytext : 0.9996156111%
p_spam_bodytext : 0.0003843888609%
you an import custom servic announc call freephon now
p_ham_bodytext : 0.7470025327%
p_spam_bodytext : 0.2529974673%
yeah whatev lol
p_ham_bodytext : 0.9621614717%
p_spam_bodytext : 0.03783852827%
today accept dayu accept me as brother ster lover dear best clos lvblefrnd jstf

In [ ]:
while True:
  message = str(input("\nEnter message: "))
  if (message != "exit!"):
    calculate_probabilities(message)
  else:
    break